<font color='red'>**Nota: Favor tomar en cuenta que los requisitos del entregable establecidos como limpieza, pre-procesamiento, estimación de covarianza, EDA y Hot Encoding fueron contemplados en el Notebook del entregable anterior y están contenidos en la misma cuenta de GitHub en la que se encuentra el presente entregable**</font>

### Carga de librerías de trabajo

Los siguientes fueron los paquetes y librerías que se utilizaron para llevar a cabo el presente análisis de datos: 

In [1]:
#Carga las librerías de trabajo básico en Python. 
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from math import sqrt

In [2]:
#Filtra las alertas de los bloques de código.
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [3]:
#Importa las librerías para normalizar el conjunto de datos.
from sklearn import preprocessing

In [4]:
#Importa las librerías algoritmos para determinar los modelos predictores
from sklearn.ensemble import RandomForestClassifier #Paquete específico para clasificación
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC #Paquete específico para clasificación

In [5]:
#Importa las liberías para realizar ingeniería de atributos
#https://towardsdatascience.com/feature-selection-in-python-recursive-feature-elimination-19f1c39b8d15
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

In [6]:
#Importa las librerías de validación cruzada
from sklearn.model_selection import train_test_split

In [7]:
#Importa las librerías de las métricas de modelado
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

### Carga de conjunto de datos

El presente bloque de código presenta la carga del conjunto de datos y los cambios de tipo de variable respectivos. Es importante mencionar que en el análisis previo se realizó **reducción dimensional** en el atributo relacionado de nivel educativo con el fin de consolidar la eduación catalogada como "otros" dentro de un mismo tipo de categoría que permitiera simplificar el conjunto de datos. 

In [8]:
#Carga el conjunto de datos procesado en el notebook de análisis.
credit = pd.read_csv("dummyVars.csv",header=0)
credit.head()

,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,MALE,FEMALE,GRAD_SCHOOL,UNIVERSITY,HIGH_SCHOOL,ED_OTHERS,MS_OTHERS,MARRIED,SINGLE,DIVORCE,PAY_0_-2,PAY_0_-1,PAY_0_0,PAY_0_1,PAY_0_2,PAY_0_3,PAY_0_4,PAY_0_5,PAY_0_6,PAY_0_7,PAY_0_8,PAY_2_-2,PAY_2_-1,PAY_2_0,PAY_2_1,PAY_2_2,PAY_2_3,PAY_2_4,PAY_2_5,PAY_2_6,PAY_2_7,PAY_2_8,PAY_3_-2,PAY_3_-1,PAY_3_0,PAY_3_1,PAY_3_2,PAY_3_3,PAY_3_4,PAY_3_5,PAY_3_6,PAY_3_7,PAY_3_8,PAY_4_-2,PAY_4_-1,PAY_4_0,PAY_4_1,PAY_4_2,PAY_4_3,PAY_4_4,PAY_4_5,PAY_4_6,PAY_4_7,PAY_4_8,PAY_5_-2,PAY_5_-1,PAY_5_0,PAY_5_2,PAY_5_3,PAY_5_4,PAY_5_5,PAY_5_6,PAY_5_7,PAY_5_8,PAY_6_-2,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3,PAY_6_4,PAY_6_5,PAY_6_6,PAY_6_7,PAY_6_8,DEFAULT_PAYMENT
0,20000,24,3913,3102,689,0,0,0,0,689,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
1,120000,26,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,90000,34,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,50000,37,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,50000,57,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [9]:
#Convierte variable dependiente en tipo categórica. 
credit['DEFAULT_PAYMENT'] = credit['DEFAULT_PAYMENT'].astype('category')

In [10]:
#Muestra la estructura del conjunto de datos
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 89 columns):
LIMIT_BAL          30000 non-null int64
AGE                30000 non-null int64
BILL_AMT1          30000 non-null int64
BILL_AMT2          30000 non-null int64
BILL_AMT3          30000 non-null int64
BILL_AMT4          30000 non-null int64
BILL_AMT5          30000 non-null int64
BILL_AMT6          30000 non-null int64
PAY_AMT1           30000 non-null int64
PAY_AMT2           30000 non-null int64
PAY_AMT3           30000 non-null int64
PAY_AMT4           30000 non-null int64
PAY_AMT5           30000 non-null int64
PAY_AMT6           30000 non-null int64
MALE               30000 non-null int64
FEMALE             30000 non-null int64
GRAD_SCHOOL        30000 non-null int64
UNIVERSITY         30000 non-null int64
HIGH_SCHOOL        30000 non-null int64
ED_OTHERS          30000 non-null int64
MS_OTHERS          30000 non-null int64
MARRIED            30000 non-null int64
SINGLE     

### Segmentación del conjunto de datos 

La presente sección muestra el código creado para la segmentación del conjunto de datos en las variables independientes y la variable dependiente a predecir. Importante mencionar que la variable dependiente es **CREDIT DEFAULT PAYMENT NEXT MONTH** renombrada como **DEFAULT_PAYMENT** para simplicidad y mejor manejo de la variable.

In [11]:
#Separa las variables independientes en el conjunto de datos
creditIndependent = credit.iloc[:,0:88]
creditIndependent.head()

,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,MALE,FEMALE,GRAD_SCHOOL,UNIVERSITY,HIGH_SCHOOL,ED_OTHERS,MS_OTHERS,MARRIED,SINGLE,DIVORCE,PAY_0_-2,PAY_0_-1,PAY_0_0,PAY_0_1,PAY_0_2,PAY_0_3,PAY_0_4,PAY_0_5,PAY_0_6,PAY_0_7,PAY_0_8,PAY_2_-2,PAY_2_-1,PAY_2_0,PAY_2_1,PAY_2_2,PAY_2_3,PAY_2_4,PAY_2_5,PAY_2_6,PAY_2_7,PAY_2_8,PAY_3_-2,PAY_3_-1,PAY_3_0,PAY_3_1,PAY_3_2,PAY_3_3,PAY_3_4,PAY_3_5,PAY_3_6,PAY_3_7,PAY_3_8,PAY_4_-2,PAY_4_-1,PAY_4_0,PAY_4_1,PAY_4_2,PAY_4_3,PAY_4_4,PAY_4_5,PAY_4_6,PAY_4_7,PAY_4_8,PAY_5_-2,PAY_5_-1,PAY_5_0,PAY_5_2,PAY_5_3,PAY_5_4,PAY_5_5,PAY_5_6,PAY_5_7,PAY_5_8,PAY_6_-2,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3,PAY_6_4,PAY_6_5,PAY_6_6,PAY_6_7,PAY_6_8
0,20000,24,3913,3102,689,0,0,0,0,689,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,120000,26,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,90000,34,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,50000,37,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,50000,57,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [12]:
#Separa la variable dependiente del conjunto de datos
creditDependent = credit.iloc[:,88:89]
creditDependent.head()

,DEFAULT_PAYMENT
0,1
1,1
2,0
3,0
4,0


### Normalización del conjunto de datos

En la presente sección se escala el conjunto de datos para que estos puedan ser interpretados más facilmente por los algoritmos encargados en entrenar los modelos predictores.

In [13]:
#Normaliza el conjunto de datos de las variables independientes
creditIndependentNormal = preprocessing.scale(creditIndependent)
print(creditIndependentNormal)

[[-1.13672015 -1.24601985 -0.64250107 ... -0.02517409 -0.03918786
  -0.00816524]
 [-0.3659805  -1.02904717 -0.65921875 ... -0.02517409 -0.03918786
  -0.00816524]
 [-0.59720239 -0.16115646 -0.29855966 ... -0.02517409 -0.03918786
  -0.00816524]
 ...
 [-1.05964618  0.16430256 -0.64722711 ... -0.02517409 -0.03918786
  -0.00816524]
 [-0.67427636  0.59824792 -0.71798186 ... -0.02517409 -0.03918786
  -0.00816524]
 [-0.90549825  1.14067961 -0.04473888 ... -0.02517409 -0.03918786
  -0.00816524]]


### Ingeniería de atributos - Eliminación Recursiva de Atributos con Validación Cruzada

En la presente sección se determinó a partir de RFECV que el número de atributos que ofrece el mayor rendimiento para un modelo predictor utilizando **Random Forest** es 88 atributos. Esto significa por tanto que no se removerán atributos del conjunto de datos dado que todos estos son necesarios para obtener la mejor predicción. 

In [14]:
RFC = RandomForestClassifier(random_state=101)

In [15]:
rfecv = RFECV(estimator = RFC, step = 1, cv=StratifiedKFold(10), scoring = 'accuracy')

In [16]:
rfecv.fit(creditIndependentNormal,creditDependent)

RFECV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
      estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                       criterion='gini', max_depth=None,
                                       max_features='auto', max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators='warn', n_jobs=None,
                                       oob_score=False, random_state=101,
                                       verbose=0, warm_start=False),
      min_features_to_select=1, n_jobs=None, scoring='accuracy', step=1,
      verbose=0)

In [17]:
#Muestra el número de atributo necesarios para obtener la mayor precisión en el modelo.
print('Optimal number of features: {}'.format(rfecv.n_features_))

Optimal number of features: 88


### Determinación de tamaño de conjuntos de datos

En la presente sección se determinan los tamaños del conjunto de datos tanto de entrenamiento como prueba. Se utilizo una proporción 70/30 respectivamente para el modelado y validación de datos.

In [18]:
#Retorna el tamaño del arreglo de variables independientes
len(creditIndependentNormal)

30000

In [19]:
#Determina el tamaño de los conjuntos de datos de entrenamiento y testeo
trainSize = int(.70*len(creditIndependentNormal))
testSize = int(.30*len(creditIndependentNormal))
print(trainSize, testSize)

21000 9000


### Segregación de conjuntos de datos para entrenamiento y prueba

In [20]:
#Establece el conjunto de datos de las variables independientes para entrenamiento.
X_training = creditIndependentNormal[0:trainSize,:]
X_training.shape

(21000, 88)

In [21]:
#Establece el conjunto de datos de las variables independientes para prueba.
X_testing = creditIndependentNormal[trainSize:len(creditIndependentNormal),:]
X_testing.shape

(9000, 88)

In [22]:
#Establece el conjunto de datos de la variable dependiente para entrenamiento.
y_training = creditDependent.iloc[0:trainSize,:]
y_training.shape

(21000, 1)

In [23]:
#Establece el conjunto de datos de la variable dependiente para prueba.
y_testing = creditDependent.iloc[trainSize:len(creditDependent),:]
y_testing.shape

(9000, 1)

### Configuración de validación cruzada

In [24]:
X_training, X_testing, y_training, y_testing = train_test_split(X_training, y_training)
X_training.shape, X_testing.shape

((15750, 88), (5250, 88))

### Entrenamiento de modelos predictores

#### Random Forest

El siguiente bloque de código contiene el modelado en el conjunto de datos de entrenamiento con la **configuración de parámetros predeterminada del algoritmo**. 

In [41]:
modelRFC01 = RandomForestClassifier()
modelRFC01.fit(X_training,y_training)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [42]:
print(cross_val_score(modelRFC01,X_training,y_training))

[0.79603885 0.80285714 0.79291294]


In [43]:
modelRFC01.score(X_training,y_training)

0.9783492063492063

A continuación se utilizan **parámetros personalizados** para el modelo predictor con el propósito de mejorar la precisión de la predicción.

In [61]:
modelRFC02 = RandomForestClassifier(n_estimators=15, max_depth=None,min_samples_split=2, random_state=0)
modelRFC02.fit(X_training,y_training)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=15,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [62]:
print(cross_val_score(modelRFC02,X_training,y_training))

[0.79584841 0.79657143 0.80167651]


In [63]:
modelRFC02.score(X_training,y_training)

0.9921269841269841

#### Support Vector Machine

El siguiente bloque de código contiene el modelado en el conjunto de datos de entrenamiento con la configuración de parámetros predeterminada del algoritmo.

In [77]:
modelSVC01 = SVC()
modelSVC01.fit(X_training,y_training)
print(cross_val_score(modelSVC01,X_training,y_training))

[0.81736812 0.80761905 0.8121547 ]


In [29]:
modelSVC01.score(X_training,y_training)

0.8241269841269842

A continuación se utilizan parámetros personalizados para el modelo predictor con el propósito de mejorar la precisión de la predicción.

In [86]:
modelSVC02 = SVC(kernel='poly',C=2.0)
modelSVC02.fit(X_training,y_training)
print(cross_val_score(modelSVC02,X_training,y_training))

[0.81527328 0.80285714 0.8121547 ]


In [89]:
modelSVC02.score(X_training,y_training)

0.8324444444444444

#### Stochastic Gradient Descent

El siguiente bloque de código contiene el modelado en el conjunto de datos de entrenamiento con la configuración de parámetros predeterminada del algoritmo.

In [71]:
modelSGDC01 = SGDClassifier()
modelSGDC01.fit(X_training,y_training)
print(cross_val_score(modelSGDC01,X_training,y_training))

[0.80460865 0.79561905 0.81329777]


In [72]:
modelSGDC01.score(X_training,y_training)

0.8125714285714286

A continuación se utilizan parámetros personalizados para el modelo predictor con el propósito de mejorar la precisión de la predicción.

In [73]:
modelSGDC02 = SGDClassifier(loss="log",penalty="elasticnet",max_iter=400,shuffle=True)
modelSGDC02.fit(X_training,y_training)
print(cross_val_score(modelSGDC2,X_training,y_training))

[0.81184536 0.8072381  0.8045342 ]


In [74]:
modelSGDC02.score(X_training,y_training)

0.8156825396825397

### Evaluación de modelos predictores

#### Random Forest

El modelo predictor que presentó el mejor rendimiendo a nivel de Random Forest fue el generado con los parámetros predeterminados del algoritmo. Es decir, el modelo **modelRFC01**.

In [70]:
predictionsRFC01 = modelRFC01.predict(X_testing)
accuracy_score(y_testing,predictionsRFC01)

0.7975238095238095

In [69]:
predictionsRFC02 = modelRFC02.predict(X_testing)
accuracy_score(y_testing,predictionsRFC02)

0.796

#### Support Vector Machine

El modelo predictor que presentó el mejor rendimiendo a nivel de Support Vector Machine fue el generado con los parámetros predeterminados del algoritmo. Es decir, el modelo **modelSVC01**.

In [88]:
predictionsSVC01 = modelSVC01.predict(X_testing)
accuracy_score(y_testing,predictionsSVC01)

0.8127619047619048

In [90]:
predictionsSVC02 = modelSVC02.predict(X_testing)
accuracy_score(y_testing,predictionsSVC02)

0.8102857142857143

#### Stochastic Gradient Descent

El modelo predictor que presentó el mejor rendimiendo a nivel de Stochastic Gradient Descent fue el generado con los parámetros personalizados en el modelo **modelSGDC02**.

In [76]:
predictionsSGDC01 = modelSGDC01.predict(X_testing)
accuracy_score(y_testing,predictionsSGDC01)

0.8083809523809524

In [75]:
predictionsSGDC02 = modelSGDC02.predict(X_testing)
accuracy_score(y_testing,predictionsSGDC02)

0.8097142857142857

### Selección de modelo predictor

Basado en la precisión obtenida de los diferentes modelos predictores y la iteración de sus parámetros de ajuste, se seleccionó el modelo predictor **Support Vector Machine** con los parámetros predeterminados de configuración.